In [1]:
import pandas as pd

In [2]:
path_to_gbm_subclone_peaks = "./GBM_Clone_Peaks/" ### TODO
path_to_cleaned_motifs = "./GBM_Clone_Cleaned_Motifs/" ### TODO
path_to_uncleaned_motifs = "./GBM_Clone_Uncleaned_Motifs/" ### TODO
modified_names_table_path = "modified_names_table.xlsx" ### TODO

In [ ]:
name_keys_df  = pd.read_excel(modified_names_table_path,dtype=str) ## TODO: get modified names mapping from publication
name_keys_df.columns = ["name","modified_name"]
name_keys_df

name_keys = dict(zip(name_keys_df["name"],name_keys_df["modified_name"]))
name_keys

In [4]:
len(set(name_keys.values()))

113

In [5]:
import scipy.stats
def perform_test(df):

    
    for idx,row in df.iterrows():
        a,b,c,d = row["cleaned_cloneA"], row["uncleaned_cloneA"], row["cleaned_cloneB"], row["uncleaned_cloneB"]
        odds_ratio = ((a/b)/(c/d))
        
        pval = scipy.stats.chi2_contingency([[a,b],[c,d]])[1]
        df.loc[idx,"odds"] = odds_ratio
        df.loc[idx,"pval_chis1"] = pval 
        
    df["pval_corrected"] = df["pval_chis1"] * len(df)
    df["pval_corrected"] = df["pval_corrected"].apply(lambda x : min(x,1.0))
    return df                              

In [6]:
df_dict = {}

for sample_name in ["GBM39","GBM45"]:
    print(sample_name)
    
    df_sample = None
    clone_name = "cloneA"
        
    peakset = pd.read_csv(f"{path_to_gbm_subclone_peaks}/{sample_name}_{clone_name}_peaks.csv")[["seqnames","start","end"]]
    
    cleaned_motifs = pd.read_csv(f"{path_to_cleaned_motifs}/ismshap_gbm_subclone_vierstrav1_{sample_name}_{clone_name}.csv")[["seqnames","start","end","group_name"]]
    cleaned_motifs["modified_names"] = cleaned_motifs["group_name"].apply(name_keys.get)
    uncleaned_motifs = pd.read_csv(f"{path_to_uncleaned_motifs}/vierstra_Archetype_{sample_name}_{clone_name}agg.csv.bed")[["seqnames","start","end","group_name"]]
    uncleaned_motifs["modified_names"] = uncleaned_motifs["group_name"].apply(name_keys.get)

    cleaned_motifs = cleaned_motifs.groupby("modified_names")["seqnames"].agg("count").reset_index(name=f"cleaned_{clone_name}")
    df_sample = cleaned_motifs.copy()
    uncleaned_motifs = uncleaned_motifs.groupby("modified_names")["seqnames"].agg("count").reset_index(name=f"uncleaned_{clone_name}")
    df_sample=df_sample.merge(uncleaned_motifs)

    clone_name = "cloneB"
    peakset = pd.read_csv(f"{path_to_gbm_subclone_peaks}/{sample_name}_{clone_name}_peaks.csv")[["seqnames","start","end"]]

    cleaned_motifs = pd.read_csv(f"{path_to_cleaned_motifs}/ismshap_gbm_subclone_vierstrav1_{sample_name}_{clone_name}.csv")[["seqnames","start","end","group_name"]]
    cleaned_motifs["modified_names"] = cleaned_motifs["group_name"].apply(name_keys.get)
    uncleaned_motifs = pd.read_csv(f"{path_to_uncleaned_motifs}/vierstra_Archetype_{sample_name}_{clone_name}agg.csv.bed")[["seqnames","start","end","group_name"]]
    uncleaned_motifs["modified_names"] = uncleaned_motifs["group_name"].apply(name_keys.get)

    cleaned_motifs = cleaned_motifs.groupby("modified_names")["seqnames"].agg("count").reset_index(name=f"cleaned_{clone_name}")
    df_sample=df_sample.merge(cleaned_motifs)
    uncleaned_motifs = uncleaned_motifs.groupby("modified_names")["seqnames"].agg("count").reset_index(name=f"uncleaned_{clone_name}")
    df_sample=df_sample.merge(uncleaned_motifs)
    
    df_sample = perform_test(df_sample.copy())
    
    df_dict[sample_name] = df_sample

GBM39
GBM45


In [ ]:
fetal39 = df_dict["GBM39"][["modified_names","odds","pval_corrected"]]
fetal39.columns = ["modified_names","gbm39_cloneA_Score","gbm39_cloneA_pval"]

fetal45 = df_dict["GBM45"][["modified_names","odds","pval_corrected"]]
fetal45.columns = ["modified_names","gbm45_cloneB_Score","gbm45_cloneB_pval"]
fetal45["gbm45_cloneB_Score"] = 1.0/fetal45["gbm45_cloneB_Score"]

fetal_df = pd.merge(fetal39, fetal45)
fetal_df


In [8]:
fetal_df["av"] = (fetal_df["gbm39_cloneA_Score"] + fetal_df["gbm45_cloneB_Score"])/2.0
fetal_df[(fetal_df["gbm39_cloneA_pval"] < 0.05) | (fetal_df["gbm45_cloneB_pval"] < 0.05)].sort_values("av",ascending=False)[:15]

,modified_names,gbm39_cloneA_Score,gbm39_cloneA_pval,gbm45_cloneB_Score,gbm45_cloneB_pval,av
25,Ebox/CAGCTG,1.330961,1.604994e-292,1.455488,0.000000e+00,1.393224
22,Ebox/CACCTG,1.256471,6.075183e-129,1.417741,7.244169e-256,1.337106
71,OSR2,1.231972,1.689987e-40,1.275536,4.617188e-49,1.253754
24,Ebox/CAGATGG,1.225033,8.310585e-125,1.262523,4.329070e-153,1.243778
39,HEN1,1.134996,7.089565e-17,1.284588,1.327729e-50,1.209792
60,MYB,1.217399,3.092000e-30,1.136243,5.208447e-13,1.176821
96,TATA,1.222413,5.840211e-04,1.084390,1.000000e+00,1.153402
70,OCT4+SOX2,1.206228,7.285343e-15,1.052521,1.000000e+00,1.129375
92,SPZ1,1.155521,6.205598e-03,1.102579,6.900089e-01,1.129050
98,TCF/LEF,1.114154,6.016190e-04,1.129258,5.965884e-05,1.121706


In [ ]:
adult39 = df_dict["GBM39"][["modified_names","odds","pval_corrected"]]
adult39.columns = ["modified_names","gbm39_cloneB_Score","gbm39_cloneB_pval"]
adult39["gbm39_cloneB_Score"] = 1.0/adult39["gbm39_cloneB_Score"]

adult45 = df_dict["GBM45"][["modified_names","odds","pval_corrected"]]
adult45.columns = ["modified_names","gbm45_cloneA_Score","gbm45_cloneA_pval"]

adult_df = pd.merge(adult39, adult45)
adult_df

In [10]:
adult_df["av"] = (adult_df["gbm39_cloneB_Score"] + adult_df["gbm45_cloneA_Score"])/2.0
adult_df[(adult_df["gbm39_cloneB_pval"] < 0.05) | (adult_df["gbm45_cloneA_pval"] < 0.05)].sort_values("av",ascending=False)[:21]

,modified_names,gbm39_cloneB_Score,gbm39_cloneB_pval,gbm45_cloneA_Score,gbm45_cloneA_pval,av
3,ARI5A,1.539893,5.195514e-05,1.602601,3.949733e-06,1.571247
99,TEAD,1.383819,3.755711e-97,1.266405,2.358104e-39,1.325112
54,MBD2,1.212024,9.462843e-30,1.280331,2.193019e-34,1.246177
83,RUNX,1.322027,6.195764e-120,1.100112,3.495944e-10,1.211069
45,HSF,1.102745,9.463709e-03,1.312820,2.349797e-24,1.207783
13,CUX,1.059506,1.000000e+00,1.324144,2.813441e-24,1.191825
17,EBF1,1.097996,1.606848e-08,1.273800,2.037318e-62,1.185898
107,ZFX,1.170344,4.231501e-89,1.198939,7.582565e-97,1.184641
49,KAISO,0.980183,1.000000e+00,1.360036,7.463276e-18,1.170109
73,P53-like,1.112945,2.280086e-12,1.210135,2.369600e-40,1.161540
